In [3]:
from langchain.agents import OpenAIFunctionsAgent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import PythonAstREPLTool
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langsmith import Client
from langchain.smith import RunEvalConfig, run_on_dataset
from pydantic import BaseModel, Field
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.agents.agent_toolkits.conversational_retrieval.tool import create_retriever_tool
from langchain.document_loaders.csv_loader import CSVLoader

In [5]:
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access environment variables
api_key = os.getenv("OPENAI_KEY")

In [19]:
# Set up a knowledge base
product_catalog = "clean_beer.csv"


pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 20)

# load product catalog
loader = CSVLoader(file_path=product_catalog)
documents = loader.load()

embedding_model = OpenAIEmbeddings(openai_api_key=api_key)
vectorstore = FAISS.from_documents(documents, embedding_model)

retriever_tool = create_retriever_tool(vectorstore.as_retriever(), "beer_name_search", "Search for a beer by name")

TEMPLATE = """You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
It is important to understand the attributes of the dataframe before working with it. This is the result of running `df.head().to_markdown()`

<df>
{dhead}
</df>

You are not meant to use only these rows to answer questions - they are meant as a way of telling you about the shape and schema of the dataframe.
You also do not have use only the information here to answer questions - you can run intermediate queries to do exporatory data analysis to give you more information as needed.

You have a tool called `beer_name_search` through which you can lookup a beer by name and find the records corresponding to beers with similar name as the query.
You should only really use this if your search term contains a beer's name. Otherwise, try to solve it with code.

For example:
    
<question>How strong is Best Beer?</question>
<logic>Use `beer_name_search` since you can use the query `Best Beer`</logic>

<question>Which beer has an ibu of 41.5</question>
<logic>Use `python_repl` since even though the question is about a beer, you don't know their name so you can't include it.</logic>
"""

class PythonInputs(BaseModel):
    query: str = Field(description="code snippet to run")



df = pd.read_csv("clean_beer.csv")
template = TEMPLATE.format(dhead=df.head().to_markdown())

prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
    ("human", "{input}")
])


repl = PythonAstREPLTool(locals={"df": df}, name="python_repl",
                         description="Runs code and returns the output of the final line",
                         args_schema=PythonInputs)
tools = [repl, retriever_tool]
agent = OpenAIFunctionsAgent(llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo", openai_api_key=api_key),
                             prompt=prompt, 
                             tools=tools,
                            )
agent_executor = AgentExecutor(agent=agent, 
                               tools=tools, 
                               max_iterations=5, 
                               early_stopping_method="generate",
                               verbose=True
                              )




In [20]:
qa_chain = get_chain()

In [23]:
agent_executor.run("which beers pair well with tacos?")



> Entering new AgentExecutor chain...

Invoking: `beer_name_search` with `tacos`


[Document(page_content="id: 116\nname: Morag's Mojito - B-Sides\ntagline: Cocktail Meets Beer.\nfirst_brewed: 08/2015\ndescription: A cunning cocktail blend of grapefruit hops, ginger spice and refreshing mint.\nimage_url: https://images.punkapi.com/v2/keg.png\nabv: 6.7\nibu: 40.0\nebc: 15.0\nsrm: 7.5\nph: 5.2\ningredients: Extra Pale, Ginger Root Citra, Citra, Citra Wyeast 1056 - American Ale™\nfood_pairing: Tamale, Salt cod croquettes, Mint cheesecake\ncontributed_by: Matt Ball\nmalts: Extra Pale, Ginger Root\nhops: Citra, Citra, Citra\nyeast: Wyeast 1056 - American Ale™", metadata={'source': 'clean_beer.csv', 'row': 115}), Document(page_content='id: 295\nname: Choco Libre\ntagline: Mexican Hot Chocolate Imperial Stout.\nfirst_brewed: 2017\ndescription: This one-off brew showcases the flavours of a Mexican spiced hot chocolate, infused with an imperial stout.\nimage_url: \nabv: 9.0\nibu: 25.0\nebc: 1

"Based on the search results, here are some beers that pair well with tacos:\n\n1. Morag's Mojito - B-Sides:\n   - ABV: 6.7%\n   - IBU: 40.0\n   - Description: A cunning cocktail blend of grapefruit hops, ginger spice, and refreshing mint.\n   - Food Pairing: Tamale, Salt cod croquettes, Mint cheesecake\n\n2. Deaf Mermaid - B-Sides:\n   - ABV: 5.2%\n   - IBU: 65.0\n   - Description: A keg-only summer special, hop-bursted and dry-hopped to infinity.\n   - Food Pairing: Grilled fish tacos, Greek salad, Goat's cheese\n\n3. Twin Atlantic:\n   - ABV: 5.0%\n   - IBU: 80.0\n   - Description: Brewed in collaboration with our AGM-headlining band, Twin Atlantic, this US style pale features some of our favorite hops and is loaded with tropical fruit.\n   - Food Pairing: Jerk Chicken Kebabs with Mango Salsa, Avocado and Citrus Salad, Pineapple Upside Down Cake\n\nPlease note that these are just a few examples, and there may be other beers that pair well with tacos as well."

In [26]:
df = pd.read_csv('clean_beer.csv')

In [29]:
df[:10].to_csv('clean_beer_test.csv', index=False)